<h1 align="center">S2R Analytics</h1>
<h2 align="center">Profitability of Client X projects: run 1</h2>

# Table of Contents

* [Part 6](#part6): Classification
    * [6.0](#6_0): Data splitting
    * [6.1](#6_1): Models
<br />
<br />
* [Part 7](#part7): Fine-tuning
* [Part 8](#part8): Ensemble learning
* [Part 9](#part9): Evaluation of the final model

## Notebook Setup

In [1]:
# Essentials
import pandas as pd
from pandas import Series, DataFrame
from pandas.api.types import CategoricalDtype
pd.options.display.max_columns = None
import sqlite3
import pyodbc
import numpy as np; np.random.seed(1)

# Image creation and display
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.patches as mpatches
from matplotlib import pyplot
import plotly.express as px
import plotly.graph_objects as go
from matplotlib.ticker import FuncFormatter
from yellowbrick.model_selection import FeatureImportances

# Preprocessing
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC

# Metrics of accuracy
from numpy import mean
from numpy import std
from sklearn import metrics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from pycm import *
import imbalanced_ensemble as imbens
from imbalanced_ensemble.ensemble.base import sort_dict_by_key
from collections import Counter

# Fine-tuning and enseble learning
from pprint import pprint
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn.base import clone
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import RandomizedSearchCV

# Other
import itertools as it
import io
import os
os.sys.path
import sys
import glob
import concurrent.futures
from __future__ import print_function
import binascii
import struct
from PIL import Image
import scipy
import scipy.misc
import scipy.cluster
import datetime, time
import functools, operator
from datetime import datetime
from numpy.random import seed
from numpy.random import randn
from numpy import percentile

In [2]:
df = pd.read_csv('../csv-files/active_projects.csv')
df.rename(columns = {'Profit Class':'Profit_Class'}, inplace=True)

## Part 6: <a class="anchor" id="part6"></a> Classification

### 6.0 <a class="anchor" id="6_0"></a> Data splitting

In [3]:
# Choose dependent variables
Y = df[['Profit_Class']]

# Drop the dependent variables from the feature data set
X = df.drop(columns = ['Profit_Class', 'Rate Group'])

# Scale the explanatory variables
X1 = pd.DataFrame(StandardScaler().fit_transform(X))
X1.columns = X.columns
X = X1

# Split data set into train and test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=1, stratify = Y)

print(f'No. of training data: {X_train.shape[0]}')
print(f'No. of training targets: {Y_train.shape[0]}')
print(f'No. of testing data: {X_test.shape[0]}')
print(f'No. of testing targets: {Y_test.shape[0]}')

No. of training data: 1706
No. of training targets: 1706
No. of testing data: 427
No. of testing targets: 427


In [4]:
for i in range(3):
    print(f'Class {i} has {(Y.Profit_Class==i).sum()} samples in full dataset.')
    print(f'Class {i} has {(Y_train.Profit_Class==i).sum()} samples in training set.')
    print(f'Class {i} has {(Y_test.Profit_Class==i).sum()} samples in test set.')

Class 0 has 410 samples in full dataset.
Class 0 has 328 samples in training set.
Class 0 has 82 samples in test set.
Class 1 has 921 samples in full dataset.
Class 1 has 737 samples in training set.
Class 1 has 184 samples in test set.
Class 2 has 802 samples in full dataset.
Class 2 has 641 samples in training set.
Class 2 has 161 samples in test set.


### 6.1 <a class="anchor" id="6_1"></a> Models

#### 6.1.1  <a class="anchor" id="6_1_1"></a> Logistic regression

In [5]:
log = LogisticRegression(random_state = 1, max_iter = 30000)
log.fit(X_train, Y_train.values.ravel())
log_y_pred=log.predict(X_test)

print('Precision score of LOG: ' + str(round(metrics.precision_score(Y_test, np.round(log_y_pred), average='weighted', zero_division=0), 3)*100)+'%')
print('F1 of LOG: ' + str(round(metrics.f1_score(Y_test, np.round(log_y_pred), average='weighted'), 3)*100)+'%')
print('Recall score of LOG: ' + str(round(metrics.recall_score(Y_test, np.round(log_y_pred), average='weighted', zero_division=0), 3)*100)+'%')
print('Accuracy score of LOG: ' + str(round(metrics.accuracy_score(Y_test, np.round(log_y_pred)), 3)*100)+'%')

Precision score of LOG: 45.0%
F1 of LOG: 44.9%
Recall score of LOG: 46.400000000000006%
Accuracy score of LOG: 46.400000000000006%


#### 6.1.2 <a class="anchor" id="6_1_2"></a> K-Neighbours classifier

In [6]:
np.random.seed(1)
knn_7 = KNeighborsClassifier(n_neighbors=7)
knn_7.fit(X_train, Y_train.values.ravel())
knn_7_y_pred = knn_7.predict(X_test)

print('Precision score of KNN-7: ' + str(round(metrics.precision_score(Y_test, np.round(knn_7_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 of KNN-7: ' + str(round(metrics.f1_score(Y_test, np.round(knn_7_y_pred), average='weighted'), 3)*100)+'%')
print('Recall score of KNN-7: ' + str(round(metrics.recall_score(Y_test, np.round(knn_7_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy score of KNN-7: ' + str(round(metrics.accuracy_score(Y_test, np.round(knn_7_y_pred)), 3)*100)+'%')

Precision score of KNN-7: 50.1%
F1 of KNN-7: 49.8%
Recall score of KNN-7: 49.6%
Accuracy score of KNN-7: 49.6%


#### 6.1.3  <a class="anchor" id="6_1_3"></a> Decision tree classifier

In [7]:
dtc = DecisionTreeClassifier(random_state = 1)
dtc = dtc.fit(X_train, Y_train.values.ravel())
dtc_y_pred = dtc.predict(X_test)

print('Precision score of DTC: ' + str(round(metrics.precision_score(Y_test, np.round(dtc_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 of DTC: ' + str(round(metrics.f1_score(Y_test, np.round(dtc_y_pred), average='weighted'), 3)*100)+'%')
print('Recall score of DTC: ' + str(round(metrics.recall_score(Y_test, np.round(dtc_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy score of DTC: ' + str(round(metrics.accuracy_score(Y_test, np.round(dtc_y_pred)), 3)*100)+'%')

Precision score of DTC: 52.2%
F1 of DTC: 51.9%
Recall score of DTC: 51.800000000000004%
Accuracy score of DTC: 51.800000000000004%


#### 6.1.4  <a class="anchor" id="6_1_4"></a> Random forest classifier

In [8]:
rfc = RandomForestClassifier(random_state = 1)
rfc.fit(X_train, Y_train.values.ravel())
rfc_y_pred=rfc.predict(X_test)

print('Precision score of RFC: ' + str(round(metrics.precision_score(Y_test, np.round(rfc_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 of RFC: ' + str(round(metrics.f1_score(Y_test, np.round(rfc_y_pred), average='weighted'), 3)*100)+'%')
print('Recall score of RFC: ' + str(round(metrics.recall_score(Y_test, np.round(rfc_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy score of RFC: ' + str(round(metrics.accuracy_score(Y_test, np.round(rfc_y_pred)), 3)*100)+'%')

Precision score of RFC: 57.099999999999994%
F1 of RFC: 57.099999999999994%
Recall score of RFC: 57.4%
Accuracy score of RFC: 57.4%


#### 6.1.5  <a class="anchor" id="6_1_5"></a> XGBoost classifier

In [9]:
xgbc = XGBClassifier(n_estimators=100, learning_rate=0.05, booster='gbtree', random_state = 1, eval_metric='mlogloss', use_label_encoder=False)
xgbc.fit(X_train, Y_train.values.ravel())
xgbc_y_pred=xgbc.predict(X_test)

print('Precision score of XGBC: ' + str(round(metrics.precision_score(Y_test, np.round(xgbc_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 of XGBC: ' + str(round(metrics.f1_score(Y_test, np.round(xgbc_y_pred), average='weighted'), 3)*100)+'%')
print('Recall score of XGBC: ' + str(round(metrics.recall_score(Y_test, np.round(xgbc_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy score of XGBC: ' + str(round(metrics.accuracy_score(Y_test, np.round(xgbc_y_pred)), 3)*100)+'%')

Precision score of XGBC: 51.7%
F1 of XGBC: 51.7%
Recall score of XGBC: 52.0%
Accuracy score of XGBC: 52.0%


#### 6.1.6  <a class="anchor" id="6_1_6"></a> Naive Bayes

In [10]:
gnb = GaussianNB()
gnb.fit(X_train, Y_train.values.ravel())
gnb_y_pred = gnb.predict(X_test)

print('Precision score of GNB: ' + str(round(metrics.precision_score(Y_test, np.round(gnb_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 of GNB: ' + str(round(metrics.f1_score(Y_test, np.round(gnb_y_pred), average='weighted'), 3)*100)+'%')
print('Recall score of GNB: ' + str(round(metrics.recall_score(Y_test, np.round(gnb_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy score of GNB: ' + str(round(metrics.accuracy_score(Y_test, np.round(gnb_y_pred)), 3)*100)+'%')

Precision score of GNB: 52.5%
F1 of GNB: 30.5%
Recall score of GNB: 40.300000000000004%
Accuracy score of GNB: 40.300000000000004%


#### 6.1.7  <a class="anchor" id="6_1_7"></a> Linear discriminant analysis

In [11]:
lda = LinearDiscriminantAnalysis(n_components = 2)
lda.fit(X_train, Y_train.values.ravel())
lda_y_pred = lda.predict(X_test)

print('Precision score of LDA: ' + str(round(metrics.precision_score(Y_test, np.round(lda_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 of LDA: ' + str(round(metrics.f1_score(Y_test, np.round(lda_y_pred), average='weighted'), 3)*100)+'%')
print('Recall score of LDA: ' + str(round(metrics.recall_score(Y_test, np.round(lda_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy score of LDA: ' + str(round(metrics.accuracy_score(Y_test, np.round(lda_y_pred)), 3)*100)+'%')

Precision score of LDA: 44.3%
F1 of LDA: 44.4%
Recall score of LDA: 45.4%
Accuracy score of LDA: 45.4%


#### 6.1.8  <a class="anchor" id="6_1_8"></a> Quadratic discriminant analysis

In [12]:
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train, Y_train.values.ravel())
qda_y_pred = qda.predict(X_test)

print('Precision score of QDA: ' + str(round(metrics.precision_score(Y_test, np.round(qda_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 of QDA: ' + str(round(metrics.f1_score(Y_test, np.round(qda_y_pred), average='weighted'), 3)*100)+'%')
print('Recall score of QDA: ' + str(round(metrics.recall_score(Y_test, np.round(qda_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy score of QDA: ' + str(round(metrics.accuracy_score(Y_test, np.round(qda_y_pred)), 3)*100)+'%')

Precision score of QDA: 48.199999999999996%
F1 of QDA: 32.0%
Recall score of QDA: 40.699999999999996%
Accuracy score of QDA: 40.699999999999996%


#### 6.1.9  <a class="anchor" id="6_1_9"></a> Ridge regression classifier

In [13]:
rdg = RidgeClassifier(alpha=1.0, random_state = 1, max_iter = 30000)
rdg.fit(X_train, Y_train.values.ravel())
rdg_y_pred=rdg.predict(X_test)

print('Precision score of RDG: ' + str(round(metrics.precision_score(Y_test, np.round(rdg_y_pred), average='weighted', zero_division=0), 3)*100)+'%')
print('F1 of RDG: ' + str(round(metrics.f1_score(Y_test, np.round(rdg_y_pred), average='weighted'), 3)*100)+'%')
print('Recall score of RDG: ' + str(round(metrics.recall_score(Y_test, np.round(rdg_y_pred), average='weighted', zero_division=0), 3)*100)+'%')
print('Accuracy score of RDG: ' + str(round(metrics.accuracy_score(Y_test, np.round(rdg_y_pred)), 3)*100)+'%')

Precision score of RDG: 46.5%
F1 of RDG: 46.300000000000004%
Recall score of RDG: 48.0%
Accuracy score of RDG: 48.0%


#### 6.1.10  <a class="anchor" id="6_1_10"></a> Support vector machines

In [14]:
svm = SVC(kernel='linear', random_state = 1, probability=True)
svm.fit(X_train, Y_train.values.ravel())
svm_y_pred = svm.predict(X_test)

print('Precision score of SVM: ' + str(round(metrics.precision_score(Y_test, np.round(svm_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 of SVM: ' + str(round(metrics.f1_score(Y_test, np.round(svm_y_pred), average='weighted'), 3)*100)+'%')
print('Recall score of SVM: ' + str(round(metrics.recall_score(Y_test, np.round(svm_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy score of SVM: ' + str(round(metrics.accuracy_score(Y_test, np.round(svm_y_pred)), 3)*100)+'%')

Precision score of SVM: 45.9%
F1 of SVM: 46.0%
Recall score of SVM: 47.8%
Accuracy score of SVM: 47.8%


## Part 7: <a class="anchor" id="part7"></a> Fine-tuning

### 7.1  <a class="anchor" id="7_1"></a> XGBoost grid search

In [15]:
# Look at parameters used by our current XGBoost model
print('Parameters currently in use:\n')
pprint(xgbc.get_params())

Parameters currently in use:

{'base_score': 0.5,
 'booster': 'gbtree',
 'callbacks': None,
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': 'mlogloss',
 'gamma': 0,
 'gpu_id': -1,
 'grow_policy': 'depthwise',
 'importance_type': None,
 'interaction_constraints': '',
 'learning_rate': 0.05,
 'max_bin': 256,
 'max_cat_to_onehot': 4,
 'max_delta_step': 0,
 'max_depth': 6,
 'max_leaves': 0,
 'min_child_weight': 1,
 'missing': nan,
 'monotone_constraints': '()',
 'n_estimators': 100,
 'n_jobs': 0,
 'num_parallel_tree': 1,
 'objective': 'multi:softprob',
 'predictor': 'auto',
 'random_state': 1,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'sampling_method': 'uniform',
 'scale_pos_weight': None,
 'subsample': 1,
 'tree_method': 'exact',
 'use_label_encoder': False,
 'validate_parameters': 1,
 'verbosity': None}


In [16]:
# Defining parameter range
xgbc_grid = {'learning_rate':[0.1],
    'n_estimators':[1000],
    'max_depth':[4,5,6],
    'min_child_weight':[6,8,10,12],
    'gamma':[i/10.0 for i in range(0,5)],
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)],
    'objective':['binary:logistic'],
    'nthread':[4],
    'seed':[1],
    'eval_metric':['mlogloss']}

pprint(xgbc_grid)

{'colsample_bytree': [0.6, 0.7, 0.8, 0.9],
 'eval_metric': ['mlogloss'],
 'gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
 'learning_rate': [0.1],
 'max_depth': [4, 5, 6],
 'min_child_weight': [6, 8, 10, 12],
 'n_estimators': [1000],
 'nthread': [4],
 'objective': ['binary:logistic'],
 'seed': [1],
 'subsample': [0.6, 0.7, 0.8, 0.9]}


In [ ]:
# Fitting the model for grid search
xgbc_tuned = GridSearchCV(XGBClassifier(), xgbc_grid, refit = True)
xgbc_tuned.fit(X_train, Y_train.values.ravel())

In [ ]:
# Print best parameter after tuning
print(xgbc_tuned.best_params_)
 
# Print how our model looks after hyper-parameter tuning
print(xgbc_tuned.best_estimator_)

In [17]:
# Create a XGBoost_tuned model
xgbc_tuned = XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.8,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='mlogloss', gamma=0.4, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_bin=256,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=6, max_leaves=0,
              min_child_weight=10, monotone_constraints='()',
              n_estimators=1000, n_jobs=4, nthread=4, num_parallel_tree=1,
              objective='multi:softprob', predictor='auto', random_state=1)

In [18]:
# Base model results
xgbc_base_y_pred = xgbc.predict(X_test)
xgbc_base_precision = round(metrics.precision_score(Y_test, np.round(xgbc_base_y_pred), average='weighted', zero_division=1), 3)*100
print('Precision of base XGBC is ' + str(xgbc_base_precision)+'%')

# Tuned model results
xgbc_tuned.fit(X_train, Y_train.values.ravel())
xgbc_tuned_y_pred = xgbc_tuned.predict(X_test)
xgbc_tuned_precision = round(metrics.precision_score(Y_test, np.round(xgbc_tuned_y_pred), average='weighted', zero_division=1), 3)*100
print('Precision of tuned XGBC is ' + str(xgbc_tuned_precision)+'%')

# Comparison
print('Improvement of {:0.2f}%'.format(100 * (xgbc_tuned_precision - xgbc_base_precision) / xgbc_base_precision))

Precision of base XGBC is 51.7%
Precision of tuned XGBC is 56.89999999999999%
Improvement of 10.06%


In [19]:
# Rest of the measures
print('F1 of tuned XGBC: ' + str(round(metrics.f1_score(Y_test, np.round(xgbc_tuned_y_pred), average='weighted'), 3)*100)+'%')
print('Recall of tuned XGBC: ' + str(round(metrics.recall_score(Y_test, np.round(xgbc_tuned_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy of tuned XGBC: ' + str(round(metrics.accuracy_score(Y_test, np.round(xgbc_tuned_y_pred)), 3)*100)+'%')

F1 of tuned XGBC: 56.8%
Recall of tuned XGBC: 56.89999999999999%
Accuracy of tuned XGBC: 56.89999999999999%


### 7.2  <a class="anchor" id="7_2"></a> Random forest classifier grid search

In [20]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto','sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
                     'max_features': max_features,
                     'max_depth': max_depth,
                     'min_samples_split': min_samples_split,
                      'min_samples_leaf': min_samples_leaf,
                      'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [100, 311, 522, 733, 944, 1155, 1366, 1577, 1788, 2000]}


In [24]:
rfc_tuned = RandomizedSearchCV(estimator = rfc,
                               param_distributions = random_grid,
                               n_iter = 100,
                               cv = 5,
                               verbose = 2,
                               random_state = 1,
                               n_jobs = -1)
                               
# Fit the random search model
rfc_tuned.fit(X_train, Y_train.values.ravel())

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=1),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 311, 522, 733,
                                                         944, 1155, 1366, 1577,
                                                         1788, 2000]},
                   random_state=1, verbose=2)

In [25]:
# Print best parameter after tuning
print(rfc_tuned.best_params_)
 
# Print how our model looks after hyper-parameter tuning
print(rfc_tuned.best_estimator_)

{'n_estimators': 1366, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 110, 'bootstrap': True}
RandomForestClassifier(max_depth=110, min_samples_split=10, n_estimators=1366,
                       random_state=1)


In [26]:
# Create an RFC_tuned model
rfc_tuned = RandomForestClassifier(max_depth=110, min_samples_split=10, n_estimators=1366, random_state=1)

In [27]:
# Base model results
rfc_base_y_pred = rfc.predict(X_test)
rfc_base_precision = round(metrics.precision_score(Y_test, np.round(rfc_base_y_pred), average='weighted', zero_division=1), 3)*100
print('Precision of base RFC is ' + str(rfc_base_precision)+'%')

# Tuned model results
rfc_tuned.fit(X_train, Y_train.values.ravel())
rfc_tuned_y_pred = rfc_tuned.predict(X_test)
rfc_tuned_precision = round(metrics.precision_score(Y_test, np.round(rfc_tuned_y_pred), average='weighted', zero_division=1), 3)*100
print('Precision of tuned RFC is ' + str(rfc_tuned_precision)+'%')

# Comparison
print('Improvement of {:0.1f}%'.format(100 * (rfc_tuned_precision - rfc_base_precision) / rfc_base_precision))

Precision of base RFC is 57.099999999999994%
Precision of tuned RFC is 56.2%
Improvement of -1.6%


In [28]:
print('F1 of tuned RFC: ' + str(round(metrics.f1_score(Y_test, np.round(rfc_tuned_y_pred), average='weighted'), 3)*100)+'%')
print('Recall score of tuned RFC: ' + str(round(metrics.recall_score(Y_test, np.round(rfc_tuned_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy score of tuned RFC: ' + str(round(metrics.accuracy_score(Y_test, np.round(rfc_tuned_y_pred)), 3)*100)+'%')

F1 of tuned RFC: 55.800000000000004%
Recall score of tuned RFC: 56.2%
Accuracy score of tuned RFC: 56.2%


### 7.3  <a class="anchor" id="7_3"></a> SVM RBF grid search

In [29]:
# Look at parameters used by our current SVM model
print('Parameters currently in use:\n')
pprint(svm.get_params())

Parameters currently in use:

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'linear',
 'max_iter': -1,
 'probability': True,
 'random_state': 1,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}


In [ ]:
# Defining parameter range
svm_grid = {'C': [0.1, 1, 2, 3, 4, 5, 10],
            'gamma': [1, 2, 3, 4, 5, 0.1, 0.01],
            'kernel': ['linear']}
 
# Fitting the model for grid search
svm_tuned = GridSearchCV(SVC(), svm_grid, refit = True) 
svm_tuned.fit(X_train, Y_train.values.ravel())

In [ ]:
# Print best parameter after tuning
print(svm_tuned.best_params_)
 
# Print how our model looks after hyper-parameter tuning
print(svm_tuned.best_estimator_)

In [31]:
# Create a tuned SVC model with linear kernel
svm_tuned = SVC(kernel='linear', C = 0.1, gamma = 1, random_state = 1, probability=True)
svm_tuned.fit(X_train, Y_train.values.ravel())

SVC(C=0.1, gamma=1, kernel='linear', probability=True, random_state=1)

In [32]:
# Base model results
svm_base_y_pred = svm.predict(X_test)
svm_base_precision = round(metrics.precision_score(Y_test, np.round(svm_base_y_pred), average='weighted', zero_division=1), 3)*100
print('Precision of base SVM is ' + str(svm_base_precision)+'%')

# Tuned model results with kernel
svm_tuned_y_pred = svm_tuned.predict(X_test)
svm_tuned_precision = round(metrics.precision_score(Y_test, np.round(svm_tuned_y_pred), average='weighted', zero_division=1), 3)*100
print('Precision of tuned SVM with RBF kernel is ' + str(svm_tuned_precision)+'%')

print('Improvement of {:0.2f}%'.format(100 * (svm_tuned_precision - svm_base_precision) / svm_base_precision))

Precision of base SVM is 45.9%
Precision of tuned SVM with RBF kernel is 45.9%
Improvement of 0.00%


In [33]:
# Rest of the measures
print('F1 of tuned SVM with RBF kernel: ' + str(round(metrics.f1_score(Y_test, np.round(svm_tuned_y_pred), average='weighted'), 3)*100)+'%')
print('Recall of tuned SVM with RBF kernel: ' + str(round(metrics.recall_score(Y_test, np.round(svm_tuned_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy of tuned SVM with RBF kernel: ' + str(round(metrics.accuracy_score(Y_test, np.round(svm_tuned_y_pred)), 3)*100)+'%')

F1 of tuned SVM with RBF kernel: 46.0%
Recall of tuned SVM with RBF kernel: 47.8%
Accuracy of tuned SVM with RBF kernel: 47.8%


In [34]:
# Create a tuned SVC model with RBF kernel
svm_tuned = SVC(kernel='rbf', C = 0.1, gamma = 1, random_state = 1, probability=True)
svm_tuned.fit(X_train, Y_train.values.ravel())

SVC(C=0.1, gamma=1, probability=True, random_state=1)

In [35]:
# Base model results
svm_base_y_pred = svm.predict(X_test)
svm_base_precision = round(metrics.precision_score(Y_test, np.round(svm_base_y_pred), average='weighted', zero_division=1), 3)*100
print('Precision of base SVM is ' + str(svm_base_precision)+'%')

# Tuned model results with kernel
svm_tuned_y_pred = svm_tuned.predict(X_test)
svm_tuned_precision = round(metrics.precision_score(Y_test, np.round(svm_tuned_y_pred), average='weighted', zero_division=1), 3)*100
print('Precision of tuned SVM with RBF kernel is ' + str(svm_tuned_precision)+'%')

print('Improvement of {:0.2f}%'.format(100 * (svm_tuned_precision - svm_base_precision) / svm_base_precision))

Precision of base SVM is 45.9%
Precision of tuned SVM with RBF kernel is 56.599999999999994%
Improvement of 23.31%


In [36]:
# Rest of the measures
print('F1 of tuned SVM with RBF kernel: ' + str(round(metrics.f1_score(Y_test, np.round(svm_tuned_y_pred), average='weighted'), 3)*100)+'%')
print('Recall of tuned SVM with RBF kernel: ' + str(round(metrics.recall_score(Y_test, np.round(svm_tuned_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy of tuned SVM with RBF kernel: ' + str(round(metrics.accuracy_score(Y_test, np.round(svm_tuned_y_pred)), 3)*100)+'%')

F1 of tuned SVM with RBF kernel: 26.400000000000002%
Recall of tuned SVM with RBF kernel: 43.1%
Accuracy of tuned SVM with RBF kernel: 43.1%


## Part 8: <a class="anchor" id="part8"></a> Ensemble learning

### 8.1  <a class="anchor" id="8_1"></a> Voting classifier

In [50]:
soft_voting = VotingClassifier(estimators=[('xgbc_t', xgbc_tuned), ('rfc_t', rfc_tuned)], voting='soft')
soft_voting.fit(X_train, Y_train.values.ravel())
sv_y_pred = soft_voting.predict(X_test)

In [51]:
print('Precision score of soft voting classifier: ' + str(round(metrics.precision_score(Y_test, np.round(sv_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 of soft voting classifier: ' + str(round(metrics.f1_score(Y_test, np.round(sv_y_pred), average='weighted'), 3)*100)+'%')
print('Recall score of soft voting classifier ' + str(round(metrics.recall_score(Y_test, np.round(sv_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy score of soft voting classifier: ' + str(round(metrics.accuracy_score(Y_test, np.round(sv_y_pred)), 3)*100)+'%')

Precision score of soft voting classifier: 57.599999999999994%
F1 of soft voting classifier: 57.199999999999996%
Recall score of soft voting classifier 57.4%
Accuracy score of soft voting classifier: 57.4%


In [45]:
hard_voting = VotingClassifier(estimators=[('xgbc_t', xgbc_tuned), ('rfc_t', rfc_tuned), ('svm_tuned', svm_tuned)], voting='hard')
hard_voting.fit(X_train, Y_train.values.ravel())
hv_y_pred = hard_voting.predict(X_test)

In [46]:
print('Precision score of hard voting classifier: ' + str(round(metrics.precision_score(Y_test, np.round(hv_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 of hard voting classifier: ' + str(round(metrics.f1_score(Y_test, np.round(hv_y_pred), average='weighted'), 3)*100)+'%')
print('Recall score of hard voting classifier ' + str(round(metrics.recall_score(Y_test, np.round(hv_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy score of hard voting classifier: ' + str(round(metrics.accuracy_score(Y_test, np.round(hv_y_pred)), 3)*100)+'%')

Precision score of hard voting classifier: 58.4%
F1 of hard voting classifier: 55.900000000000006%
Recall score of hard voting classifier 56.699999999999996%
Accuracy score of hard voting classifier: 56.699999999999996%


### 8.2  <a class="anchor" id="8_2"></a> Stacking

#### 8.2.1  <a class="anchor" id="8_2_1"></a> All models

In [83]:
def get_stacking():
    level11 = list()
    level11.append(('hard voting', hard_voting))
    level11.append(('rfc', rfc))
    level11.append(('xgbc tuned', xgbc_tuned))
    level11.append(('svm tuned', svm_tuned))
    level11.append(('gnb', gnb))
    level11.append(('dtc', dtc))
    level11.append(('knn', knn_7))
    level11.append(('qda', qda))
    level11.append(('rdg', rdg))
    level11.append(('log', log))
    level11.append(('lda', lda))
    model = StackingClassifier(estimators=level11, final_estimator=hard_voting, cv=5)
    return model

In [84]:
level11 = list()
level11.append(('hard voting', hard_voting))
level11.append(('rfc', rfc))
level11.append(('xgbc tuned', xgbc_tuned))
level11.append(('svm tuned', svm_tuned))
level11.append(('gnb', gnb))
level11.append(('dtc', dtc))
level11.append(('knn', knn_7))
level11.append(('qda', qda))
level11.append(('rdg', rdg))
level11.append(('log', log))
level11.append(('lda', lda))
level11.append(('stacking', get_stacking()))

In [85]:
stack11_hv = StackingClassifier(estimators=level11, final_estimator=hard_voting, cv=5)
stack11_hv = stack11_hv.fit(X, Y.values.ravel())
stack11_hv_y_pred = stack11_hv.predict(X_test)

c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are c

In [86]:
print('Precision with 11 models learnt on HV: ' + str(round(metrics.precision_score(Y_test, np.round(stack11_hv_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 with 11 models learnt on HV: ' + str(round(metrics.f1_score(Y_test, np.round(stack11_hv_y_pred), average='weighted'), 3)*100)+'%')
print('Recall with 11 models learnt on HV: ' + str(round(metrics.recall_score(Y_test, np.round(stack11_hv_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy with 11 models learnt on HV: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack11_hv_y_pred)), 3)*100)+'%')

Precision with 11 models learnt on HV: 48.1%
F1 with 11 models learnt on HV: 41.9%
Recall with 11 models learnt on HV: 46.800000000000004%
Accuracy with 11 models learnt on HV: 46.800000000000004%


In [87]:
def get_stacking():
    level11 = list()
    level11.append(('hard voting', hard_voting))
    level11.append(('rfc', rfc))
    level11.append(('xgbc tuned', xgbc_tuned))
    level11.append(('svm tuned', svm_tuned))
    level11.append(('gnb', gnb))
    level11.append(('dtc', dtc))
    level11.append(('knn', knn_7))
    level11.append(('qda', qda))
    level11.append(('rdg', rdg))
    level11.append(('log', log))
    level11.append(('lda', lda))
    model = StackingClassifier(estimators=level11, final_estimator=rfc, cv=5)
    return model

In [88]:
stack11_rfc = StackingClassifier(estimators=level11, final_estimator=rfc, cv=5)
stack11_rfc = stack11_rfc.fit(X, Y.values.ravel())
stack11_rfc_y_pred = stack11_rfc.predict(X_test)

c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are c

In [89]:
print('Precision with 11 models learnt on base RFC: ' + str(round(metrics.precision_score(Y_test, np.round(stack11_rfc_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 with 11 models learnt on base RFC: ' + str(round(metrics.f1_score(Y_test, np.round(stack11_rfc_y_pred), average='weighted'), 3)*100)+'%')
print('Recall with 11 models learnt on base RFC: ' + str(round(metrics.recall_score(Y_test, np.round(stack11_rfc_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy with 11 models learnt on base RFC: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack11_rfc_y_pred)), 3)*100)+'%')

Precision with 11 models learnt on base RFC: 47.699999999999996%
F1 with 11 models learnt on base RFC: 46.1%
Recall with 11 models learnt on base RFC: 47.099999999999994%
Accuracy with 11 models learnt on base RFC: 47.099999999999994%


In [90]:
def get_stacking():
    level11 = list()
    level11.append(('hard voting', hard_voting))
    level11.append(('rfc', rfc))
    level11.append(('xgbc tuned', xgbc_tuned))
    level11.append(('svm tuned', svm_tuned))
    level11.append(('gnb', gnb))
    level11.append(('dtc', dtc))
    level11.append(('knn', knn_7))
    level11.append(('qda', qda))
    level11.append(('rdg', rdg))
    level11.append(('log', log))
    level11.append(('lda', lda))
    model = StackingClassifier(estimators=level11, final_estimator=xgbc_tuned, cv=5)
    return model

In [91]:
stack11_xgbc_t = StackingClassifier(estimators=level11, final_estimator=xgbc_tuned, cv=5)
stack11_xgbc_t = stack11_xgbc_t.fit(X, Y.values.ravel())
stack11_xgbc_t_y_pred = stack11_xgbc_t.predict(X_test)

c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are c

In [92]:
print('Precision with 11 models learnt on tuned XGBC: ' + str(round(metrics.precision_score(Y_test, np.round(stack11_xgbc_t_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 with 11 models learnt on tuned XGBC: ' + str(round(metrics.f1_score(Y_test, np.round(stack11_xgbc_t_y_pred), average='weighted'), 3)*100)+'%')
print('Recall with 11 models learnt on tuned XGBC: ' + str(round(metrics.recall_score(Y_test, np.round(stack11_xgbc_t_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy with 11 models learnt on tuned XGBC: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack11_xgbc_t_y_pred)), 3)*100)+'%')

Precision with 11 models learnt on tuned XGBC: 46.800000000000004%
F1 with 11 models learnt on tuned XGBC: 43.9%
Recall with 11 models learnt on tuned XGBC: 46.800000000000004%
Accuracy with 11 models learnt on tuned XGBC: 46.800000000000004%


#### 8.2.2  <a class="anchor" id="8_2_2"></a> Top 10 models

In [95]:
def get_stacking():
    level10 = list()
    level10.append(('hard voting', hard_voting))
    level10.append(('rfc', rfc))
    level10.append(('xgbc tuned', xgbc_tuned))
    level10.append(('svm tuned', svm_tuned))
    level10.append(('gnb', gnb))
    level10.append(('dtc', dtc))
    level10.append(('knn', knn_7))
    level10.append(('qda', qda))
    level10.append(('rdg', rdg))
    level10.append(('log', log))
    model = StackingClassifier(estimators=level10, final_estimator=hard_voting, cv=5)
    return model

In [96]:
level10 = list()
level10.append(('hard voting', hard_voting))
level10.append(('rfc', rfc))
level10.append(('xgbc tuned', xgbc_tuned))
level10.append(('svm tuned', svm_tuned))
level10.append(('gnb', gnb))
level10.append(('dtc', dtc))
level10.append(('knn', knn_7))
level10.append(('qda', qda))
level10.append(('rdg', rdg))
level10.append(('log', log))
level10.append(('stacking', get_stacking()))

In [97]:
stack10_hv = StackingClassifier(estimators=level10, final_estimator=hard_voting, cv=5)
stack10_hv = stack10_hv.fit(X, Y.values.ravel())
stack10_hv_y_pred = stack10_hv.predict(X_test)

c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are c

In [98]:
print('Precision with 10 models learnt on HV: ' + str(round(metrics.precision_score(Y_test, np.round(stack10_hv_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 with 10 models learnt on HV: ' + str(round(metrics.f1_score(Y_test, np.round(stack10_hv_y_pred), average='weighted'), 3)*100)+'%')
print('Recall with 10 models learnt on HV: ' + str(round(metrics.recall_score(Y_test, np.round(stack10_hv_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy with 10 models learnt on HV: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack10_hv_y_pred)), 3)*100)+'%')

Precision with 10 models learnt on HV: 52.400000000000006%
F1 with 10 models learnt on HV: 45.5%
Recall with 10 models learnt on HV: 50.1%
Accuracy with 10 models learnt on HV: 50.1%


In [100]:
def get_stacking():
    level10 = list()
    level10.append(('hard voting', hard_voting))
    level10.append(('rfc', rfc))
    level10.append(('xgbc tuned', xgbc_tuned))
    level10.append(('svm tuned', svm_tuned))
    level10.append(('gnb', gnb))
    level10.append(('dtc', dtc))
    level10.append(('knn', knn_7))
    level10.append(('qda', qda))
    level10.append(('rdg', rdg))
    level10.append(('log', log))
    model = StackingClassifier(estimators=level10, final_estimator=rfc, cv=5)
    return model

In [101]:
stack10_rfc = StackingClassifier(estimators=level10, final_estimator=rfc, cv=5)
stack10_rfc = stack10_rfc.fit(X, Y.values.ravel())
stack10_rfc_y_pred = stack10_rfc.predict(X_test)

c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are c

In [102]:
print('Precision with 10 models learnt on base RFC: ' + str(round(metrics.precision_score(Y_test, np.round(stack10_rfc_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 with 10 models learnt on base RFC: ' + str(round(metrics.f1_score(Y_test, np.round(stack10_rfc_y_pred), average='weighted'), 3)*100)+'%')
print('Recall with 10 models learnt on base RFC: ' + str(round(metrics.recall_score(Y_test, np.round(stack10_rfc_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy with 10 models learnt on base RFC: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack10_rfc_y_pred)), 3)*100)+'%')

Precision with 10 models learnt on base RFC: 47.5%
F1 with 10 models learnt on base RFC: 44.3%
Recall with 10 models learnt on base RFC: 47.3%
Accuracy with 10 models learnt on base RFC: 47.3%


In [103]:
def get_stacking():
    level10 = list()
    level10.append(('hard voting', hard_voting))
    level10.append(('rfc', rfc))
    level10.append(('xgbc tuned', xgbc_tuned))
    level10.append(('svm tuned', svm_tuned))
    level10.append(('gnb', gnb))
    level10.append(('dtc', dtc))
    level10.append(('knn', knn_7))
    level10.append(('qda', qda))
    level10.append(('rdg', rdg))
    level10.append(('log', log))
    model = StackingClassifier(estimators=level10, final_estimator=xgbc_tuned, cv=5)
    return model

In [104]:
stack10_xgbc_t = StackingClassifier(estimators=level10, final_estimator=xgbc_tuned, cv=5)
stack10_xgbc_t = stack10_xgbc_t.fit(X, Y.values.ravel())
stack10_xgbc_t_y_pred = stack10_xgbc_t.predict(X_test)

c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are c

In [105]:
print('Precision with 10 models learnt on tuned XGBC: ' + str(round(metrics.precision_score(Y_test, np.round(stack10_xgbc_t_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 with 10 models learnt on tuned XGBC: ' + str(round(metrics.f1_score(Y_test, np.round(stack10_xgbc_t_y_pred), average='weighted'), 3)*100)+'%')
print('Recall with 10 models learnt on tuned XGBC: ' + str(round(metrics.recall_score(Y_test, np.round(stack10_xgbc_t_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy with 10 models learnt on tuned XGBC: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack10_xgbc_t_y_pred)), 3)*100)+'%')

Precision with 10 models learnt on tuned XGBC: 53.6%
F1 with 10 models learnt on tuned XGBC: 49.9%
Recall with 10 models learnt on tuned XGBC: 52.5%
Accuracy with 10 models learnt on tuned XGBC: 52.5%


#### 8.2.3  <a class="anchor" id="8_2_3"></a> Top 9 models

In [108]:
def get_stacking():
    level9 = list()
    level9.append(('hard voting', hard_voting))
    level9.append(('rfc', rfc))
    level9.append(('xgbc tuned', xgbc_tuned))
    level9.append(('svm tuned', svm_tuned))
    level9.append(('gnb', gnb))
    level9.append(('dtc', dtc))
    level9.append(('knn', knn_7))
    level9.append(('qda', qda))
    level9.append(('rdg', rdg))
    model = StackingClassifier(estimators=level9, final_estimator=hard_voting, cv=5)
    return model

In [109]:
level9 = list()
level9.append(('hard voting', hard_voting))
level9.append(('rfc', rfc))
level9.append(('xgbc tuned', xgbc_tuned))
level9.append(('svm tuned', svm_tuned))
level9.append(('gnb', gnb))
level9.append(('dtc', dtc))
level9.append(('knn', knn_7))
level9.append(('qda', qda))
level9.append(('rdg', rdg))
level9.append(('stacking', get_stacking()))

In [110]:
stack9_hv = StackingClassifier(estimators=level9, final_estimator=hard_voting, cv=5)
stack9_hv = stack9_hv.fit(X, Y.values.ravel())
stack9_hv_y_pred = stack9_hv.predict(X_test)

c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are c

In [111]:
print('Precision with 9 models learnt on HV: ' + str(round(metrics.precision_score(Y_test, np.round(stack9_hv_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 with 9 models learnt on HV: ' + str(round(metrics.f1_score(Y_test, np.round(stack9_hv_y_pred), average='weighted'), 3)*100)+'%')
print('Recall with 9 models learnt on HV: ' + str(round(metrics.recall_score(Y_test, np.round(stack9_hv_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy with 9 models learnt on HV: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack9_hv_y_pred)), 3)*100)+'%')

Precision with 9 models learnt on HV: 48.1%
F1 with 9 models learnt on HV: 41.9%
Recall with 9 models learnt on HV: 47.3%
Accuracy with 9 models learnt on HV: 47.3%


In [112]:
def get_stacking():
    level9 = list()
    level9.append(('hard voting', hard_voting))
    level9.append(('rfc', rfc))
    level9.append(('xgbc tuned', xgbc_tuned))
    level9.append(('svm tuned', svm_tuned))
    level9.append(('gnb', gnb))
    level9.append(('dtc', dtc))
    level9.append(('knn', knn_7))
    level9.append(('qda', qda))
    level9.append(('rdg', rdg))
    model = StackingClassifier(estimators=level9, final_estimator=rfc, cv=5)
    return model

In [113]:
stack9_rfc = StackingClassifier(estimators=level9, final_estimator=rfc, cv=5)
stack9_rfc = stack9_rfc.fit(X, Y.values.ravel())
stack9_rfc_y_pred = stack9_rfc.predict(X_test)

c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are c

In [114]:
print('Precision with 9 models learnt on base RFC: ' + str(round(metrics.precision_score(Y_test, np.round(stack9_rfc_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 with 9 models learnt on base RFC: ' + str(round(metrics.f1_score(Y_test, np.round(stack9_rfc_y_pred), average='weighted'), 3)*100)+'%')
print('Recall with 9 models learnt on base RFC: ' + str(round(metrics.recall_score(Y_test, np.round(stack9_rfc_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy with 9 models learnt on base RFC: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack9_rfc_y_pred)), 3)*100)+'%')

Precision with 9 models learnt on base RFC: 47.4%
F1 with 9 models learnt on base RFC: 44.7%
Recall with 9 models learnt on base RFC: 47.3%
Accuracy with 9 models learnt on base RFC: 47.3%


In [115]:
def get_stacking():
    level9 = list()
    level9.append(('hard voting', hard_voting))
    level9.append(('rfc', rfc))
    level9.append(('xgbc tuned', xgbc_tuned))
    level9.append(('svm tuned', svm_tuned))
    level9.append(('gnb', gnb))
    level9.append(('dtc', dtc))
    level9.append(('knn', knn_7))
    level9.append(('qda', qda))
    level9.append(('rdg', rdg))
    model = StackingClassifier(estimators=level9, final_estimator=xgbc_tuned, cv=5)
    return model

In [116]:
stack9_xgbc_t = StackingClassifier(estimators=level9, final_estimator=xgbc_tuned, cv=5)
stack9_xgbc_t = stack9_xgbc_t.fit(X, Y.values.ravel())
stack9_xgbc_t_y_pred = stack9_xgbc_t.predict(X_test)

c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are c

In [117]:
print('Precision with 9 models learnt on tuned XGBC: ' + str(round(metrics.precision_score(Y_test, np.round(stack9_xgbc_t_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 with 9 models learnt on tuned XGBC: ' + str(round(metrics.f1_score(Y_test, np.round(stack9_xgbc_t_y_pred), average='weighted'), 3)*100)+'%')
print('Recall with 9 models learnt on tuned XGBC: ' + str(round(metrics.recall_score(Y_test, np.round(stack9_xgbc_t_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy with 9 models learnt on tuned XGBC: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack9_xgbc_t_y_pred)), 3)*100)+'%')

Precision with 9 models learnt on tuned XGBC: 48.199999999999996%
F1 with 9 models learnt on tuned XGBC: 44.3%
Recall with 9 models learnt on tuned XGBC: 47.5%
Accuracy with 9 models learnt on tuned XGBC: 47.5%


#### 8.2.4  <a class="anchor" id="8_2_4"></a> Top 8 models

In [120]:
def get_stacking():
    level8 = list()
    level8.append(('hard voting', hard_voting))
    level8.append(('rfc', rfc))
    level8.append(('xgbc tuned', xgbc_tuned))
    level8.append(('svm tuned', svm_tuned))
    level8.append(('gnb', gnb))
    level8.append(('dtc', dtc))
    level8.append(('knn', knn_7))
    level8.append(('qda', qda))
    model = StackingClassifier(estimators=level8, final_estimator=hard_voting, cv=5)
    return model

In [121]:
level8 = list()
level8.append(('hard voting', hard_voting))
level8.append(('rfc', rfc))
level8.append(('xgbc tuned', xgbc_tuned))
level8.append(('svm tuned', svm_tuned))
level8.append(('gnb', gnb))
level8.append(('dtc', dtc))
level8.append(('knn', knn_7))
level8.append(('qda', qda))
level8.append(('stacking', get_stacking()))

In [122]:
stack8_hv = StackingClassifier(estimators=level8, final_estimator=hard_voting, cv=5)
stack8_hv = stack8_hv.fit(X, Y.values.ravel())
stack8_hv_y_pred = stack8_hv.predict(X_test)

c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are c

In [123]:
print('Precision with 8 models learnt on HV: ' + str(round(metrics.precision_score(Y_test, np.round(stack8_hv_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 with 8 models learnt on HV: ' + str(round(metrics.f1_score(Y_test, np.round(stack8_hv_y_pred), average='weighted'), 3)*100)+'%')
print('Recall with 8 models learnt on HV: ' + str(round(metrics.recall_score(Y_test, np.round(stack8_hv_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy with 8 models learnt on HV: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack8_hv_y_pred)), 3)*100)+'%')

Precision with 8 models learnt on HV: 56.2%
F1 with 8 models learnt on HV: 50.5%
Recall with 8 models learnt on HV: 54.300000000000004%
Accuracy with 8 models learnt on HV: 54.300000000000004%


In [124]:
def get_stacking():
    level8 = list()
    level8.append(('hard voting', hard_voting))
    level8.append(('rfc', rfc))
    level8.append(('xgbc tuned', xgbc_tuned))
    level8.append(('svm tuned', svm_tuned))
    level8.append(('gnb', gnb))
    level8.append(('dtc', dtc))
    level8.append(('knn', knn_7))
    level8.append(('qda', qda))
    model = StackingClassifier(estimators=level8, final_estimator=rfc, cv=5)
    return model

In [125]:
stack8_rfc = StackingClassifier(estimators=level8, final_estimator=rfc, cv=5)
stack8_rfc = stack8_rfc.fit(X, Y.values.ravel())
stack8_rfc_y_pred = stack8_rfc.predict(X_test)

c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are c

In [126]:
print('Precision with 8 models learnt on base RFC: ' + str(round(metrics.precision_score(Y_test, np.round(stack8_rfc_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 with 8 models learnt on base RFC: ' + str(round(metrics.f1_score(Y_test, np.round(stack8_rfc_y_pred), average='weighted'), 3)*100)+'%')
print('Recall with 8 models learnt on base RFC: ' + str(round(metrics.recall_score(Y_test, np.round(stack8_rfc_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy with 8 models learnt on base RFC: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack8_rfc_y_pred)), 3)*100)+'%')

Precision with 8 models learnt on base RFC: 53.7%
F1 with 8 models learnt on base RFC: 50.8%
Recall with 8 models learnt on base RFC: 53.900000000000006%
Accuracy with 8 models learnt on base RFC: 53.900000000000006%


In [127]:
def get_stacking():
    level8 = list()
    level8.append(('hard voting', hard_voting))
    level8.append(('rfc', rfc))
    level8.append(('xgbc tuned', xgbc_tuned))
    level8.append(('svm tuned', svm_tuned))
    level8.append(('gnb', gnb))
    level8.append(('dtc', dtc))
    level8.append(('knn', knn_7))
    level8.append(('qda', qda))
    model = StackingClassifier(estimators=level8, final_estimator=xgbc_tuned, cv=5)
    return model

In [128]:
stack8_xgbc_t = StackingClassifier(estimators=level8, final_estimator=xgbc_tuned, cv=5)
stack8_xgbc_t = stack8_xgbc_t.fit(X, Y.values.ravel())
stack8_xgbc_t_y_pred = stack8_xgbc_t.predict(X_test)

c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\MarfaPopova\S2R Analytics\Development & Support Team - Power BI for Synergy - Advanced Analytics\DataFlowExtract\venv\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are c

In [129]:
print('Precision with 8 models learnt on tuned XGBC: ' + str(round(metrics.precision_score(Y_test, np.round(stack8_xgbc_t_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 with 8 models learnt on tuned XGBC: ' + str(round(metrics.f1_score(Y_test, np.round(stack8_xgbc_t_y_pred), average='weighted'), 3)*100)+'%')
print('Recall with 8 models learnt on tuned XGBC: ' + str(round(metrics.recall_score(Y_test, np.round(stack8_xgbc_t_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy with 8 models learnt on tuned XGBC: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack8_xgbc_t_y_pred)), 3)*100)+'%')

Precision with 8 models learnt on tuned XGBC: 54.300000000000004%
F1 with 8 models learnt on tuned XGBC: 52.400000000000006%
Recall with 8 models learnt on tuned XGBC: 54.1%
Accuracy with 8 models learnt on tuned XGBC: 54.1%


#### 8.2.5  <a class="anchor" id="8_2_5"></a> Top 7 models

In [ ]:
def get_stacking():
    level9 = list()
    level9.append(('hard voting', hard_voting))
    level9.append(('rfc', rfc))
    level9.append(('xgbc tuned', xgbc_tuned))
    level9.append(('svm tuned', svm_tuned))
    level9.append(('gnb', gnb))
    level9.append(('dtc', dtc))
    level9.append(('knn', knn_7))
    level9.append(('qda', qda))
    level9.append(('rdg', rdg))
    model = StackingClassifier(estimators=level9, final_estimator=hard_voting, cv=5)
    return model

In [ ]:
level9 = list()
level9.append(('hard voting', hard_voting))
level9.append(('rfc', rfc))
level9.append(('xgbc tuned', xgbc_tuned))
level9.append(('svm tuned', svm_tuned))
level9.append(('gnb', gnb))
level9.append(('dtc', dtc))
level9.append(('knn', knn_7))
level9.append(('qda', qda))
level9.append(('rdg', rdg))
level9.append(('stacking', get_stacking()))

In [ ]:
stack9_hv = StackingClassifier(estimators=level9, final_estimator=hard_voting, cv=5)
stack9_hv = stack9_hv.fit(X, Y.values.ravel())
stack9_hv_y_pred = stack9_hv.predict(X_test)

In [ ]:
print('Precision with 9 models learnt on HV: ' + str(round(metrics.precision_score(Y_test, np.round(stack9_hv_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 with 9 models learnt on HV: ' + str(round(metrics.f1_score(Y_test, np.round(stack9_hv_y_pred), average='weighted'), 3)*100)+'%')
print('Recall with 9 models learnt on HV: ' + str(round(metrics.recall_score(Y_test, np.round(stack9_hv_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy with 9 models learnt on HV: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack9_hv_y_pred)), 3)*100)+'%')

In [ ]:
def get_stacking():
    level9 = list()
    level9.append(('hard voting', hard_voting))
    level9.append(('rfc', rfc))
    level9.append(('xgbc tuned', xgbc_tuned))
    level9.append(('svm tuned', svm_tuned))
    level9.append(('gnb', gnb))
    level9.append(('dtc', dtc))
    level9.append(('knn', knn_7))
    level9.append(('qda', qda))
    level9.append(('rdg', rdg))
    model = StackingClassifier(estimators=level9, final_estimator=rfc, cv=5)
    return model

In [ ]:
stack9_rfc = StackingClassifier(estimators=level9, final_estimator=rfc, cv=5)
stack9_rfc = stack9_rfc.fit(X, Y.values.ravel())
stack9_rfc_y_pred = stack9_rfc.predict(X_test)

In [ ]:
print('Precision with 9 models learnt on base RFC: ' + str(round(metrics.precision_score(Y_test, np.round(stack9_rfc_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 with 9 models learnt on base RFC: ' + str(round(metrics.f1_score(Y_test, np.round(stack9_rfc_y_pred), average='weighted'), 3)*100)+'%')
print('Recall with 9 models learnt on base RFC: ' + str(round(metrics.recall_score(Y_test, np.round(stack9_rfc_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy with 9 models learnt on base RFC: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack9_rfc_y_pred)), 3)*100)+'%')

In [ ]:
def get_stacking():
    level9 = list()
    level9.append(('hard voting', hard_voting))
    level9.append(('rfc', rfc))
    level9.append(('xgbc tuned', xgbc_tuned))
    level9.append(('svm tuned', svm_tuned))
    level9.append(('gnb', gnb))
    level9.append(('dtc', dtc))
    level9.append(('knn', knn_7))
    level9.append(('qda', qda))
    level9.append(('rdg', rdg))
    model = StackingClassifier(estimators=level9, final_estimator=xgbc_tuned, cv=5)
    return model

In [ ]:
stack9_xgbc_t = StackingClassifier(estimators=level9, final_estimator=xgbc_tuned, cv=5)
stack9_xgbc_t = stack9_xgbc_t.fit(X, Y.values.ravel())
stack9_xgbc_t_y_pred = stack9_xgbc_t.predict(X_test)

In [ ]:
print('Precision with 9 models learnt on tuned XGBC: ' + str(round(metrics.precision_score(Y_test, np.round(stack9_xgbc_t_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 with 9 models learnt on tuned XGBC: ' + str(round(metrics.f1_score(Y_test, np.round(stack9_xgbc_t_y_pred), average='weighted'), 3)*100)+'%')
print('Recall with 9 models learnt on tuned XGBC: ' + str(round(metrics.recall_score(Y_test, np.round(stack9_xgbc_t_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy with 9 models learnt on tuned XGBC: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack9_xgbc_t_y_pred)), 3)*100)+'%')

#### 8.2.6  <a class="anchor" id="8_2_6"></a> Top 6 models

#### 8.2.7  <a class="anchor" id="8_2_7"></a> Top 5 models

#### 8.2.8  <a class="anchor" id="8_2_8"></a> Top 4 models

#### 8.2.9  <a class="anchor" id="8_2_9"></a> Top 3 models

In [71]:
def get_stacking():
    level3 = list()
    level3.append(('hard voting', hard_voting))
    level3.append(('rfc', rfc))
    level3.append(('xgbc tuned', xgbc_tuned))
    model = StackingClassifier(estimators=level3, final_estimator=hard_voting, cv=5)
    return model

In [72]:
level3 = list()
level3.append(('hard voting', hard_voting))
level3.append(('rfc', rfc))
level3.append(('xgbc tuned', xgbc_tuned))
level3.append(('stacking', get_stacking()))

In [73]:
stack3_hv = StackingClassifier(estimators=level3, final_estimator=hard_voting, cv=5)
stack3_hv = stack3_hv.fit(X, Y.values.ravel())
stack3_hv_y_pred = stack3_hv.predict(X_test)

In [74]:
print('Precision with 3 models learnt on HV: ' + str(round(metrics.precision_score(Y_test, np.round(stack3_hv_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 with 3 models learnt on HV: ' + str(round(metrics.f1_score(Y_test, np.round(stack3_hv_y_pred), average='weighted'), 3)*100)+'%')
print('Recall with 3 models learnt on HV: ' + str(round(metrics.recall_score(Y_test, np.round(stack3_hv_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy with 3 models learnt on HV: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack3_hv_y_pred)), 3)*100)+'%')

Precision with 3 models learnt on HV: 65.7%
F1 with 3 models learnt on HV: 63.2%
Recall with 3 models learnt on HV: 63.0%
Accuracy with 3 models learnt on HV: 63.0%


In [75]:
def get_stacking():
    level3 = list()
    level3.append(('hard voting', hard_voting))
    level3.append(('rfc', rfc))
    level3.append(('xgbc tuned', xgbc_tuned))
    model = StackingClassifier(estimators=level3, final_estimator=rfc, cv=5)
    return model

In [76]:
stack3_rfc = StackingClassifier(estimators=level3, final_estimator=rfc, cv=5)
stack3_rfc = stack3_rfc.fit(X, Y.values.ravel())
stack3_rfc_y_pred = stack3_rfc.predict(X_test)

In [77]:
print('Precision with 3 models learnt on base RFC: ' + str(round(metrics.precision_score(Y_test, np.round(stack3_rfc_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 with 3 models learnt on base RFC: ' + str(round(metrics.f1_score(Y_test, np.round(stack3_rfc_y_pred), average='weighted'), 3)*100)+'%')
print('Recall with 3 models learnt on base RFC: ' + str(round(metrics.recall_score(Y_test, np.round(stack3_rfc_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy with 3 models learnt on base RFC: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack3_rfc_y_pred)), 3)*100)+'%')

Precision with 3 models learnt on base RFC: 59.3%
F1 with 3 models learnt on base RFC: 58.3%
Recall with 3 models learnt on base RFC: 58.099999999999994%
Accuracy with 3 models learnt on base RFC: 58.099999999999994%


In [78]:
def get_stacking():
    level3 = list()
    level3.append(('hard voting', hard_voting))
    level3.append(('rfc', rfc))
    level3.append(('xgbc tuned', xgbc_tuned))
    model = StackingClassifier(estimators=level3, final_estimator=xgbc_tuned, cv=5)
    return model

In [79]:
stack3_xgbc_t = StackingClassifier(estimators=level3, final_estimator=xgbc_tuned, cv=5)
stack3_xgbc_t = stack3_xgbc_t.fit(X, Y.values.ravel())
stack3_xgbc_t_y_pred = stack3_xgbc_t.predict(X_test)

In [80]:
print('Precision with 3 models learnt on tuned XGBC: ' + str(round(metrics.precision_score(Y_test, np.round(stack3_xgbc_t_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 with 3 models learnt on tuned XGBC: ' + str(round(metrics.f1_score(Y_test, np.round(stack3_xgbc_t_y_pred), average='weighted'), 3)*100)+'%')
print('Recall with 3 models learnt on tuned XGBC: ' + str(round(metrics.recall_score(Y_test, np.round(stack3_xgbc_t_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy with 3 models learnt on tuned XGBC: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack3_xgbc_t_y_pred)), 3)*100)+'%')

Precision with 3 models learnt on tuned XGBC: 58.8%
F1 with 3 models learnt on tuned XGBC: 57.9%
Recall with 3 models learnt on tuned XGBC: 57.599999999999994%
Accuracy with 3 models learnt on tuned XGBC: 57.599999999999994%


#### 8.2.10  <a class="anchor" id="8_2_10"></a> Top 2 models

In [54]:
def get_stacking():
    level2 = list()
    level2.append(('hard voting', hard_voting))
    level2.append(('rfc', rfc))
    model = StackingClassifier(estimators=level2, final_estimator=hard_voting, cv=5)
    return model

In [55]:
level2 = list()
level2.append(('hard voting', hard_voting))
level2.append(('rfc', rfc))
level2.append(('stacking', get_stacking()))

In [57]:
stack2_hv = StackingClassifier(estimators=level2, final_estimator=hard_voting, cv=5)
stack2_hv = stack2_hv.fit(X, Y.values.ravel())
stack2_hv_y_pred = stack2_hv.predict(X_test)

In [58]:
print('Precision with 2 models learnt on HV: ' + str(round(metrics.precision_score(Y_test, np.round(stack2_hv_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 with 2 models learnt on HV: ' + str(round(metrics.f1_score(Y_test, np.round(stack2_hv_y_pred), average='weighted'), 3)*100)+'%')
print('Recall with 2 models learnt on HV: ' + str(round(metrics.recall_score(Y_test, np.round(stack2_hv_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy with 2 models learnt on HV: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack2_hv_y_pred)), 3)*100)+'%')

Precision with 2 models learnt on HV: 67.9%
F1 with 2 models learnt on HV: 56.8%
Recall with 2 models learnt on HV: 58.8%
Accuracy with 2 models learnt on HV: 58.8%


In [59]:
def get_stacking():
    level2 = list()
    level2.append(('hard voting', hard_voting))
    level2.append(('rfc', rfc))
    model = StackingClassifier(estimators=level2, final_estimator=rfc, cv=5)
    return model

In [60]:
stack2_rfc = StackingClassifier(estimators=level2, final_estimator=rfc, cv=5)
stack2_rfc = stack2_rfc.fit(X, Y.values.ravel())
stack2_rfc_y_pred = stack2_rfc.predict(X_test)

In [61]:
print('Precision with 2 models learnt on base RFC: ' + str(round(metrics.precision_score(Y_test, np.round(stack2_rfc_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 with 2 models learnt on base RFC: ' + str(round(metrics.f1_score(Y_test, np.round(stack2_rfc_y_pred), average='weighted'), 3)*100)+'%')
print('Recall with 2 models learnt on base RFC: ' + str(round(metrics.recall_score(Y_test, np.round(stack2_rfc_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy with 2 models learnt on base RFC: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack2_rfc_y_pred)), 3)*100)+'%')

Precision with 2 models learnt on base RFC: 56.2%
F1 with 2 models learnt on base RFC: 54.50000000000001%
Recall with 2 models learnt on base RFC: 54.300000000000004%
Accuracy with 2 models learnt on base RFC: 54.300000000000004%


In [64]:
def get_stacking():
    level2 = list()
    level2.append(('hard voting', hard_voting))
    level2.append(('rfc', rfc))
    model = StackingClassifier(estimators=level2, final_estimator=xgbc_tuned, cv=5)
    return model

In [65]:
stack2_xgbc_t = StackingClassifier(estimators=level2, final_estimator=xgbc_tuned, cv=5)
stack2_xgbc_t = stack2_xgbc_t.fit(X, Y.values.ravel())
stack2_xgbc_t_y_pred = stack2_xgbc_t.predict(X_test)

In [66]:
print('Precision with 2 models learnt on tuned XGBC: ' + str(round(metrics.precision_score(Y_test, np.round(stack2_xgbc_t_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 with 2 models learnt on tuned XGBC: ' + str(round(metrics.f1_score(Y_test, np.round(stack2_xgbc_t_y_pred), average='weighted'), 3)*100)+'%')
print('Recall with 2 models learnt on tuned XGBC: ' + str(round(metrics.recall_score(Y_test, np.round(stack2_xgbc_t_y_pred), average='weighted'), 3)*100)+'%')
print('Accuracy with 2 models learnt on tuned XGBC: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack2_xgbc_t_y_pred)), 3)*100)+'%')

Precision with 2 models learnt on tuned XGBC: 60.199999999999996%
F1 with 2 models learnt on tuned XGBC: 54.800000000000004%
Recall with 2 models learnt on tuned XGBC: 55.50000000000001%
Accuracy with 2 models learnt on tuned XGBC: 55.50000000000001%
